# Imports

In [1]:
import sys
import os

# In Jupyter notebooks, __file__ is not defined. Use workaround:
if "__file__" in globals():
    base_dir = os.path.dirname(__file__)

else:
    base_dir = os.getcwd()
    sys.path.insert(0, os.path.abspath(os.path.join(base_dir, '..', '..')))


from app.agents.real_estate_agent import real_state_agent
from app.models.user_models import UserInput
from pydantic_ai.messages import ToolCallPart
import duckdb

# Sandbox

# Agent testing

In [21]:
connection = duckdb.connect("../../db/db.duckdb")

## Real estate agent

## Free tests

In [3]:
user_input = "Ola, pode consultar os horários disponíveis para o imóvel de id 1190f6a63fa2754842367bc615f8ac1c"

async def test_real_state_agent():
    response = await real_state_agent.run(user_input, deps=UserInput(connection=connection, user_name="Pedro"))
    print(response)

response = await test_real_state_agent()
print(response)

ModelHTTPError: status_code: 400, model_name: llama-3.3-70b-versatile, body: {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=final_result>{"properties":null,"response":"Ol\\u00e1 Pedro! Aqui est\\u00e3o os hor\\u00e1rios dispon\\u00edveis para o im\\u00f3vel de id 1190f6a63fa2754842367bc615f8ac1c: \\n| property_id                      | slot_start                | slot_end                  | status   |\\n|:---------------------------------|:--------------------------|:--------------------------|:---------|\\n| 1190f6a63fa2754842367bc615f8ac1c | 2025-07-10 08:00:00-03:00 | 2025-07-10 08:30:00-03:00 | free     |\\n| 1190f6a63fa2754842367bc615f8ac1c | 2025-07-10 08:30:00-03:00 | 2025-07-10 09:00:00-03:00 | free     |\\n| 1190f6a63fa2754842367bc615f8ac1c | 2025-07-10 09:30:00-03:00 | 2025-07-10 10:00:00-03:00 | free     |\\n| 1

In [4]:
params = {"bairro":"boqueirao","cidade":"Curitiba","n_quartos":2,"preco_max":500000}
n_quartos = params.get('n_quartos')
preco_max = params.get('preco_max')
cidade = params.get('cidade')
bairro = params.get('bairro')

In [6]:
base_query = "SELECT property_id, preco, tamanho, cidade, bairro, rua, n_quartos, n_banheiros, n_garagem FROM properties"
filters = []
params = {}

if preco_max is not None:
    filters.append(f"preco <= {preco_max}")
    params["preco_max"] = preco_max
if n_quartos is not None:
    filters.append(f"n_quartos = {n_quartos}")
    params["n_quartos"] = n_quartos
if cidade is not None:
    filters.append(f"cidade ILIKE '%{cidade.lower()}%'")
    params["cidade"] = f"%{cidade.lower()}%"
    if bairro is not None:
        filters.append(f"bairro ILIKE '%{bairro.lower()}%'")
        params["bairro"] = f"%{bairro.lower()}%"

if filters:
    base_query += " WHERE " + " AND ".join(filters)

df = connection.execute(base_query).fetch_df()
df

,property_id,preco,tamanho,cidade,bairro,rua,n_quartos,n_banheiros,n_garagem
0,1190f6a63fa2754842367bc615f8ac1c,385000.0,70.0,curitiba,alto boqueirao,Rua Irmã Maria Francisca Michel,2,2,1
1,7c45661eaff5997da3c0359a1a037d83,120000.0,40.0,curitiba,alto boqueirao,"Travessa Célia Cardozo Dos Santos, 137",2,2,3
2,a1d999dfa8484de37ee579d014d56556,392000.0,58.0,curitiba,boqueirao,Rua Zonardy Ribas,2,2,1


## Tests

### 1

In [ ]:
user_input = "Hi there!"

tool_names = []
custom_tools = ["search_properties", "get_property_slots", "book_property_slot", "cancel_property_slot"]
async with real_state_agent.iter(user_input, deps=UserInput(connection=connection, user_name="Alex")) as agent_run:
    async for node in agent_run:
        if real_state_agent.is_call_tools_node(node):
            for tool_call in node.model_response.parts:
                if isinstance(tool_call, ToolCallPart) :
                    tool_names.append(tool_call.tool_name)

not any(tool in tool_names for tool in custom_tools)

False

### 2

In [4]:
user_input = "Quais os horários para visitar o imóvel?"

tool_names = []
async with real_state_agent.iter(user_input, deps=UserInput(connection=connection, user_name="Alex")) as agent_run:
    async for node in agent_run:
        if real_state_agent.is_call_tools_node(node):
            for tool_call in node.model_response.parts:
                if isinstance(tool_call, ToolCallPart) :
                    tool_names.append(tool_call.tool_name)

### 3

In [ ]:
user_input = "Gostaria de agendar uma visita para o imóvel 'abcfoo42' no dia 2024-12-25 as 10 da manhã."
#user_input = "Quero agendar uma visita para o imovel de id '32ih7c' as 10 da manhã."

tool_names = []
async with real_state_agent.iter(user_input, deps=UserInput(connection=connection, user_name="Alex")) as agent_run:
    async for node in agent_run:
        if real_state_agent.is_call_tools_node(node):
            for tool_call in node.model_response.parts:
                if isinstance(tool_call, ToolCallPart):
                    tool_names.append(tool_call)